# Fetch list of all ports for a given bounding box

In [1]:
from aisdb.ports.api import WorldPortIndexClient # for weather

# Initialize the client
client = WorldPortIndexClient()

# Query Gulf of St. Lawrence region
# df_ports = client.fetch_ports(ymin, ymax, xmin, xmax, out_path="gulf_ports.csv") # save out to a file
df_ports = client.fetch_ports(
    lat_min=45.0,
    lat_max=51.5,
    lon_min=-71.5,
    lon_max=-55.0
)
# Filter for cargo-capable ports
df_cargo = client.filter_by_cargo_depth(df_ports)

Querying WPI with bounds: LATITUDE >= 45.0 AND LATITUDE <= 51.5 AND LONGITUDE >= -71.5 AND LONGITUDE <= -55.0
Retrieved 61 ports
2 ports with cargo depth in ('A', 'B', 'C', 'D', 'E', 'F')


In [2]:
df_ports.head(5)

,OBJECTID,INDEX_NO,REGION_NO,PORT_NAME,COUNTRY,LATITUDE,LONGITUDE,PUB,CHART,HARBORSIZE,...,FUEL_OIL,DIESEL,DECKSUPPLY,ENG_SUPPLY,REPAIRCODE,DRYDOCK,RAILWAY,GlobalID,LAT,LON
0,1428,6170,6070,GOLDSBORO,CA,45.183333,-61.650000,145,14100,V,...,,Y,,,,,,a7c4e30c-bf5e-48b7-bfe8-ed51ba5e81c5,45.183333,-61.650000
1,1429,6190,6070,STORMONT,CA,45.216667,-61.716667,145,14100,V,...,,,,,,,,e3cdd0ac-50d0-458a-9237-f621793dc4e1,45.216667,-61.716667
2,1430,6220,6070,SONORA,CA,45.066667,-61.916667,145,14100,V,...,,,,,,,,1d79427d-7193-494c-9c25-9ba414a5e134,45.066667,-61.916667
3,1438,6500,6070,WINDSOR,CA,45.000000,-64.150000,145,14040,V,...,Y,Y,,,C,,,0eb1ef80-d7c9-4b5d-9474-04ebcbee2b18,45.000000,-64.150000
4,1439,6550,6070,ST JOHN,CA,45.266667,-66.050000,145,14044,M,...,Y,Y,Y,Y,A,L,,75723b12-d5dd-4ba7-9136-5e6da2a56aff,45.266667,-66.050000


# Get H3 indexes for the ports

In [3]:
from aisdb.discretize.h3 import Discretizer

# Initialize discretizer
discretizer = Discretizer(resolution=6)

# Rename and cast lat/lon columns
df_ports = df_ports.rename(columns={"LAT": "lat", "LON": "lon"})
df_ports[["lat", "lon"]] = df_ports[["lat", "lon"]].astype(float)

# Apply get_h3_index row-wise
df_ports["h3_index_res_6"] = df_ports.apply(lambda row: discretizer.get_h3_index(row["lat"], row["lon"]), axis=1)

In [4]:
print(df_ports[["PORT_NAME","lat", "lon", "h3_index_res_6","HARBORSIZE"]].head(10))

         PORT_NAME        lat        lon   h3_index_res_6 HARBORSIZE
0        GOLDSBORO  45.183333 -61.650000  862b2a167ffffff          V
1         STORMONT  45.216667 -61.716667  862b2a177ffffff          V
2           SONORA  45.066667 -61.916667  862b2a057ffffff          V
3          WINDSOR  45.000000 -64.150000  862b016f7ffffff          V
4          ST JOHN  45.266667 -66.050000  862b1ca2fffffff          M
5  PORT BAYSIDE NB  45.150000 -67.133333  862b1c60fffffff          V
6            CANSO  45.333333 -61.000000  862b2859fffffff          V
7      DOUGLASTOWN  48.766667 -64.383333  862b30207ffffff          V
8         CHANDLER  48.366667 -64.666667  862b3149fffffff          S
9         CARAQUET  47.800000 -64.933333  862b06d9fffffff          S


In [5]:
import plotly.express as px

# Ensure port_name exists
df_ports = df_ports.rename(columns={"PORT_NAME": "port_name"}) if "PORT_NAME" in df_ports.columns else df_ports.assign(port_name="Port")

# Add sailing anchor emoji to hover
df_ports["hover_label"] = df_ports["port_name"].apply(lambda x: f"⚓ {x}")

# Assign each port a unique color
fig = px.scatter_map(
    df_ports,
    lat="lat",
    lon="lon",
    hover_name="hover_label",
    color="port_name",  # different color for each port
    zoom=5,
    height=700
)

fig.update_layout(
    mapbox_style="open-street-map",  # or "stamen-terrain", "carto-positron", etc.
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    legend_title_text="⚓ Ports"
)

fig.show()
